# JAVIS Fine-tuning (Google Colab)

Qwen2.5-7B-Instruct QLoRA 파인튜닝

**필요사항:**
- Google Colab (무료 T4 GPU 또는 유료 A100)
- HuggingFace 토큰 (Qwen 모델 다운로드용)
- 학습 데이터 (JSONL)

## 1. 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# 패키지 설치
!pip install -q torch transformers datasets peft trl bitsandbytes accelerate scipy

In [ ]:
# HuggingFace 로그인
from huggingface_hub import login

# 여기에 HuggingFace 토큰 입력
HF_TOKEN = "hf_xxxxxxxxxxxxxxxxxxxxx"  # @param {type:"string"}
login(token=HF_TOKEN)

## 2. 학습 데이터 업로드

In [ ]:
# Google Drive 마운트 (데이터 업로드용)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 또는 직접 업로드
from google.colab import files

print("conversations_xxx.jsonl 파일을 업로드하세요")
uploaded = files.upload()

# 업로드된 파일명
TRAINING_DATA = list(uploaded.keys())[0]
print(f"업로드된 파일: {TRAINING_DATA}")

## 3. 모델 및 데이터 로드

In [ ]:
import json
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# 설정
BASE_MODEL = "Qwen/Qwen2.5-7B-Instruct"
OUTPUT_DIR = "./javis-adapter"

# 하이퍼파라미터
EPOCHS = 3
BATCH_SIZE = 2  # T4는 메모리 작으니까 2
LEARNING_RATE = 2e-4
LORA_R = 64
LORA_ALPHA = 16
MAX_SEQ_LENGTH = 1024  # T4는 1024로

In [ ]:
# 데이터 로드
def load_training_data(path):
    conversations = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                conversations.append(json.loads(line))
    return Dataset.from_list(conversations)

dataset = load_training_data(TRAINING_DATA)
print(f"로드된 대화 수: {len(dataset)}")

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 데이터 포맷팅
def format_conversation(example):
    text = tokenizer.apply_chat_template(
        example['messages'],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

dataset = dataset.map(format_conversation, remove_columns=dataset.column_names)
print("데이터 포맷팅 완료")
print(f"예시:\n{dataset[0]['text'][:500]}...")

In [ ]:
# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# 모델 로드
print("모델 로딩 중... (몇 분 걸림)")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False
print("모델 로드 완료")

In [ ]:
# LoRA 설정
peft_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

trainable, total = model.get_nb_trainable_parameters()
print(f"학습 파라미터: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")

## 4. 학습 실행

In [ ]:
# 학습 설정
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=8,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    gradient_checkpointing=True,  # 메모리 절약
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    tokenizer=tokenizer,
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_text_field="text",
)

print("학습 시작!")
trainer.train()

## 5. 모델 저장

In [ ]:
# Adapter 저장
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Adapter 저장 완료: {OUTPUT_DIR}")

In [ ]:
# 메타데이터 저장
from datetime import datetime

metadata = {
    "base_model": BASE_MODEL,
    "created_at": datetime.now().isoformat(),
    "training_config": {
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "lora_r": LORA_R,
        "lora_alpha": LORA_ALPHA,
    },
    "dataset_size": len(dataset),
}

with open(f"{OUTPUT_DIR}/metadata.json", 'w') as f:
    json.dump(metadata, f, indent=2)

print("메타데이터 저장 완료")

In [ ]:
# 파일 목록 확인
!ls -la {OUTPUT_DIR}

## 6. 다운로드 또는 HuggingFace 업로드

In [ ]:
# 로컬로 다운로드
!zip -r javis-adapter.zip {OUTPUT_DIR}
files.download('javis-adapter.zip')

In [ ]:
# 또는 HuggingFace Hub에 업로드
HF_USERNAME = "your-username"  # @param {type:"string"}
REPO_NAME = "javis-adapter-v1"  # @param {type:"string"}

model.push_to_hub(f"{HF_USERNAME}/{REPO_NAME}")
tokenizer.push_to_hub(f"{HF_USERNAME}/{REPO_NAME}")

print(f"업로드 완료: https://huggingface.co/{HF_USERNAME}/{REPO_NAME}")

## 7. 테스트 추론

In [ ]:
# 파인튜닝된 모델로 테스트
from peft import PeftModel

# Base 모델 다시 로드
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Adapter 적용
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

# 테스트
messages = [
    {"role": "user", "content": "안녕, 넌 누구야?"}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7, do_sample=True)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("응답:")
print(response)